In [1]:
import pandas as pd
import numpy as np
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import ssl
import tqdm

In [2]:
#header for metadata
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [3]:
url5 = "https://housing.com/in/buy/mumbai/mumbai"

In [4]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [5]:
page = requests.get(url5, headers=headers)#, proxies=proxies)
content = page.content
soup = BeautifulSoup(content)
places_list = []
for tag in soup.find_all('a'):
  link = tag.get("href")
  searchd = "/in/buy/mumbai"
  if link.find(searchd) != -1:
    new_link = "https://housing.com" + link
    places_list.append(new_link)
print(places_list)
Total_list5=[]
for i in places_list:
  page = requests.get(i, headers=headers)#, proxies=proxies)
  content = page.content
  soup = BeautifulSoup(content)
  link_list = []
  for tag in soup.find_all('a'):
    link = tag.get("href")
    searchd = "/in/buy/projects/page"
    if link.find(searchd) != -1:
      new_link = "https://housing.com" + link
      link_list.append(new_link)
  Total_list5.extend(link_list)
print(Total_list5)

['https://housing.com/in/buy/mumbai/fort', 'https://housing.com/in/buy/mumbai/borivali_west_1', 'https://housing.com/in/buy/mumbai/mulund_west', 'https://housing.com/in/buy/mumbai/virar_east', 'https://housing.com/in/buy/mumbai/mulund_east', 'https://housing.com/in/buy/mumbai/mira_road_east', 'https://housing.com/in/buy/mumbai/santacruz_east', 'https://housing.com/in/buy/mumbai/mulund_west', 'https://housing.com/in/buy/mumbai/mahim_west', 'https://housing.com/in/buy/mumbai/vasant_nagar', 'https://housing.com/in/buy/mumbai/gokul_garden_vwx', 'https://housing.com/in/buy/mumbai/d_n_nagar', 'https://housing.com/in/buy/mumbai/pirojshanagar', 'https://housing.com/in/buy/mumbai/subhash_nagar_bhandup_west', 'https://housing.com/in/buy/mumbai/tulshet_pada', 'https://housing.com/in/buy/mumbai/apollo_bandar', 'https://housing.com/in/buy/mumbai/industrial_area_bhandup_west', 'https://housing.com/in/buy/mumbai/cuffe_parade', 'https://housing.com/in/buy/mumbai/tagore_road', 'https://housing.com/in/b

In [6]:
def extract_data(url):
  word="Sizes.css-ltuii1::after{content:'\\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}}"
  page = requests.get(url, headers=headers)#, proxies=proxies)
  content = page.content
  soup = BeautifulSoup(content)
  #print(soup)
  result = {}
  flat_data = soup.findAll('div',attrs={'class' : 'css-j2zgcq'})
  flat_name = flat_data[0].find('h1',attrs={'class':'css-1hidc9c'})
  if flat_name is not None:
    flat_name = flat_name.text
  else:
    flat_name = "Flat name not mentioned"
  result['Flat Name'] = flat_name
  flat_address = flat_data[0].find('div',attrs={'class':'css-1ty5xzi'})
  if flat_address is not None:
    flat_address = flat_address.text
  else:
    flat_address = "Flat address not mentioned"
  result['Flat address'] = flat_address
  key = []
  value = []
  word2="Sizes"
  for data in soup.findAll('section',attrs={'class' : 'css-13dph6'}):
    for i in data.findAll('div',attrs={'class' : 'css-0'}):
      if i.text == word:
        key.append(word2)
      else:
        key.append(i.text)
    for j in data.findAll('div',attrs={'class' : 'css-1k19e3'}):
      value.append(j.text)
  for i in range(0,len(value)):
    result[key[i]] = value[i]
  tables = soup.find_all("table")
  table = tables[0]
  key_data=[]
  value_data=[]
  #tab_data = [[cell.text for cell in row.find_all(["th","td"])] for row in table.find_all("tr")]
  for row in table.find_all("tr"):
    for cell in row.find_all("th"):
      key_data.append(cell.text)
    for cell in row.find_all("td"):
      for data in cell.find_all("div"):
        value_data.append(data.text)
  for i in range(0,len(value_data)-1):
    result[key_data[i]] = value_data[i]
  data = soup.findAll("div",attrs={"class":"css-1vyxx8y"})
  for i in data:
    result["About"]=i.text
  images = soup.find_all('img')
  images_list = []
  for image in images: 
    if image['src'].startswith("https://"):
      images_list.append(image['src'])
  result["images"]=images_list
  return result

In [7]:
total_dict5=[]
for i in range(0,110):
  print(Total_list5[i])
  total_dict5.append(extract_data(Total_list5[i]))
print(total_dict5)

https://housing.com/in/buy/projects/page/264499-aplite-greenstone-heritage-by-aplite-group-in-fort
https://housing.com/in/buy/projects/page/271315-dosti-tulip-dosti-west-county-by-dosti-realty-in-thane-west?crf_track=%257B%2522placement%2522%253A%2522pdp_np_fsl%2522%252C%2522adId%2522%253A6323650%257D&fltcnt=53b345ef-3b68-4e6a-965e-f3b0031b3b99
https://housing.com/in/buy/projects/page/264499-aplite-greenstone-heritage-by-aplite-group-in-fort?fltcnt=16b2d6ff-7642-4e7a-b15f-c42e01deb41b
https://housing.com/in/buy/projects/page/264499-aplite-greenstone-heritage-by-aplite-group-in-fort
https://housing.com/in/buy/projects/page/264499-aplite-greenstone-heritage-by-aplite-group-in-fort
https://housing.com/in/buy/projects/page/264499-aplite-greenstone-heritage-by-aplite-group-in-fort
https://housing.com/in/buy/projects/page/51094-monte-south-1-by-marathon-realty-adani-group-in-byculla
https://housing.com/in/buy/projects/page/133526-one-marina-by-valencia-and-mishal-ventures-pvt-ltd-in-marine-l

In [8]:
data = pd.DataFrame(total_dict5)
data.head(10)

,Flat Name,Flat address,Configurations,Possession Starts,Avg. Price,Sizes,About,images,Project Area,Project Size,Launch Date,Configuration,Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}},Size,Possession Status
0,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dosti Tulip Dosti West County,"Thane West, Thane","2, 3 BHK Apartments","Oct, 2026",Price on request,580.00 sq.ft. - 923.00 sq.ft.,"Property for sale in Thane West, Thane. This p...",[https://is1-2.housingcdn.com/4f2250e8/be2da2c...,2.77 Acres,3 Buildings - 658 units,"Jan, 2022",NaN,NaN,NaN,NaN
2,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Monte South 1,"Plot No. 179, Khatau Mills Compund Near Y Brid...","2, 2.5, 3.5 BHK Apartments","Jun, 2022",41.35 K/sq.ft,856.00 sq.ft. - 1699.00 sq.ft.,Montesouth 1 is a residential venture at Bycul...,[https://is1-3.housingcdn.com/4f2250e8/0b8001a...,NaN,1 Building - 168 units,"Dec, 2013",NaN,NaN,NaN,NaN
7,One Marina,"Chira Bazar Chandanwadi, Mumbai South , Mumbai","2, 3, 4, 5 BHK Apartments",NaN,53.49 K/sq.ft,860.00 sq.ft. - 4517.00 sq.ft.,Valencia And Mishal One Marina is a residentia...,[https://is1-2.housingcdn.com/012c1500/71d34e5...,NaN,1 Building,NaN,NaN,NaN,NaN,NaN
8,Greenstone Heritage,"Ramabai Ambedkar Marg, Near Hotel New Bengal, ...",NaN,NaN,Price on request,NaN,"Greenstone Heritage in Fort, Mumbai is a resid...",[https://is1-2.housingcdn.com/4f2250e8/c847bbd...,NaN,NaN,NaN,BHK,On Request,On Request,NaN
9,Aplite Greenstone Heritage Wing D,"CS No. 1492, Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Oct, 2024",30.55 K/sq.ft,384.00 sq.ft. - 671.00 sq.ft.,There are various amenities in Beauty Greensto...,[https://is1-2.housingcdn.com/4f2250e8/ff6f3c1...,NaN,1 Building - 92 units,NaN,NaN,NaN,NaN,NaN


In [9]:
data["Configurations"]=data['Configurations'].combine_first(data['Configuration'])

In [10]:
data["Sizes"]=data['Sizes'].combine_first(data['Size'])

In [11]:
data=data.rename(columns = {'Possession Starts':'Possession_Starts','Possession Status':'Possession_Status'})

In [12]:
data["Possession_Starts"]=data['Possession_Starts'].combine_first(data['Possession_Status'])

In [13]:
data.head(10)

,Flat Name,Flat address,Configurations,Possession_Starts,Avg. Price,Sizes,About,images,Project Area,Project Size,Launch Date,Configuration,Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}},Size,Possession_Status
0,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dosti Tulip Dosti West County,"Thane West, Thane","2, 3 BHK Apartments","Oct, 2026",Price on request,580.00 sq.ft. - 923.00 sq.ft.,"Property for sale in Thane West, Thane. This p...",[https://is1-2.housingcdn.com/4f2250e8/be2da2c...,2.77 Acres,3 Buildings - 658 units,"Jan, 2022",NaN,NaN,NaN,NaN
2,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Aplite Greenstone Heritage,"Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Apr, 2024",Price on request,384.00 sq.ft. - 671.00 sq.ft.,Aplite Greenstone Heritage is a project by Apl...,[https://is1-2.housingcdn.com/4f2250e8/83f1616...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Monte South 1,"Plot No. 179, Khatau Mills Compund Near Y Brid...","2, 2.5, 3.5 BHK Apartments","Jun, 2022",41.35 K/sq.ft,856.00 sq.ft. - 1699.00 sq.ft.,Montesouth 1 is a residential venture at Bycul...,[https://is1-3.housingcdn.com/4f2250e8/0b8001a...,NaN,1 Building - 168 units,"Dec, 2013",NaN,NaN,NaN,NaN
7,One Marina,"Chira Bazar Chandanwadi, Mumbai South , Mumbai","2, 3, 4, 5 BHK Apartments",NaN,53.49 K/sq.ft,860.00 sq.ft. - 4517.00 sq.ft.,Valencia And Mishal One Marina is a residentia...,[https://is1-2.housingcdn.com/012c1500/71d34e5...,NaN,1 Building,NaN,NaN,NaN,NaN,NaN
8,Greenstone Heritage,"Ramabai Ambedkar Marg, Near Hotel New Bengal, ...",BHK,NaN,Price on request,On Request,"Greenstone Heritage in Fort, Mumbai is a resid...",[https://is1-2.housingcdn.com/4f2250e8/c847bbd...,NaN,NaN,NaN,BHK,On Request,On Request,NaN
9,Aplite Greenstone Heritage Wing D,"CS No. 1492, Fort, Mumbai South , Mumbai","1, 2 BHK Apartments","Oct, 2024",30.55 K/sq.ft,384.00 sq.ft. - 671.00 sq.ft.,There are various amenities in Beauty Greensto...,[https://is1-2.housingcdn.com/4f2250e8/ff6f3c1...,NaN,1 Building - 92 units,NaN,NaN,NaN,NaN,NaN


In [14]:
data = data.drop(columns=['Configuration', 'Possession_Status','Size',"Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}}"])

In [16]:
data.to_csv('mumbai.csv', index=True)

In [17]:
data.shape

(110, 11)